In [ ]:
QUESTION 1

In [ ]:
(base) kabiromohd@DESKTOP-NSHTE4G:~$ curl localhost:9200
{
  "name" : "24766db9cb61",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "I20k5wIhTjikx39YoYBbaw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}

In [ ]:
QUESTION 1 --- Answer: "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73"

In [ ]:
QUESTION 2

In [1]:
!pip install requests

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
!pip install elasticsearch

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [ ]:
QUESTION 2 ---- Answer: index

In [ ]:
QUESTION 3

In [4]:
from elasticsearch import Elasticsearch

In [5]:
es_client = Elasticsearch('http://localhost:9200')

In [8]:
es_client.info()

ObjectApiResponse({'name': 'a6d57a39d5c8', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ZnrqgJI4T2W5aLUrvC-qUQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
from tqdm.auto import tqdm

/home/kabiromohd/.local/share/virtualenvs/llmzoomcamp-N5KHg8Se/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████| 948/948 [04:18<00:00,  3.67it/s]


In [10]:
query = "How do I execute a command in a running docker container?"

In [20]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text",],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [21]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_score'])
result_docs

[84.050095, 75.54128, 72.08518]

In [ ]:
 QUESTION 3 --- Answer: 84.05

In [ ]:
QUESTION 4

In [13]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text",],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [14]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])
result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [ ]:
QUESTION 4 ---- ANSWER: How do I copy files from a different folder into docker container’s working directory?

In [ ]:
QUESTION 5

In [22]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text",],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [23]:
def build_prompt(query, search_results):
           
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.
        
        QUESTION: {question}
        
        CONTEXT:
        {context}
        """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [24]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    return prompt

In [25]:
result = rag(query)
result

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\n        Use only the facts from the CONTEXT when answering the QUESTION.\n        \n        QUESTION: How do I execute a command in a running docker container?\n        \n        CONTEXT:\n        question: How do I debug a docker container?\nanswer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nquestion: How do I copy files from my local machine to docker container?\nanswer: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command

In [26]:
len(result)

1546

In [ ]:
QUESTION 5 ---- Answer: 1462

In [ ]:
QUESTION 6

In [29]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 94.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.2 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.0/785.0 kB 1.8 MB/s eta 0:00:0000:0100:01


In [27]:
import tiktoken

In [28]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [29]:
tokens = encoding.encode(result)
num_tokens = len(tokens)
print(f"Number of tokens: {num_tokens}")


Number of tokens: 328


In [ ]:
QUESTION 6 ----- Answer: 322

In [ ]:
QUESTION 7

In [30]:
import ollama
import openai
from openai import OpenAI

In [31]:
def llm(prompt):
    client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
    )
    
    response = client.chat.completions.create(
    model='phi3',
    messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [33]:
def rag2(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [34]:
rag2_result = rag2(query)
rag2_result

' If the container is already running, execute a command in the specific container by using the `docker exec` command:\ndocker exec -it <container-id> bash\nThis will allow you to execute commands directly within the running docker container.'